<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/ComputerVision/DeepLearning/MiniVGGNet_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeNet - Rede Neural Convolucional - Classificação de Imagens - Cifar10

## Leitura de dados

In [1]:
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/ComputerVision/DeepLearning/convnet.py


--2019-04-19 01:54:15--  https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/ComputerVision/DeepLearning/convnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6344 (6.2K) [text/plain]
Saving to: ‘convnet.py’

convnet.py          100%[===================>]   6.20K  --.-KB/s    in 0s      

2019-04-19 01:54:15 (146 MB/s) - ‘convnet.py’ saved [6344/6344]



In [0]:
!rm  images.zip

In [40]:
!curl -L -o imagens.zip https://github.com/vladimiralencar/DeepLearning-LANA/raw/master/ComputerVision/DeepLearning/imagens.zip?raw=true
!unzip imagens.zip 
!rm imagens/.DS_Store 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   180  100   180    0     0    782      0 --:--:-- --:--:-- --:--:--   782
100  503k  100  503k    0     0  1389k      0 --:--:-- --:--:-- --:--:-- 1389k
Archive:  imagens.zip
replace imagens/image8.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: imagens/image8.jpg      
  inflating: imagens/image9.jpg      
  inflating: imagens/image10.jpg     
  inflating: imagens/image7.jpg      
  inflating: imagens/image6.jpg      
  inflating: imagens/image4.jpg      
  inflating: imagens/image5.jpg      
  inflating: imagens/image1.jpg      
  inflating: imagens/image2.jpg      
  inflating: imagens/image3.jpg      


In [43]:
!rm imagens/.DS_Store 
!ls -ila imagens

rm: cannot remove 'imagens/.DS_Store': No such file or directory
total 556
 5637305 drwxr-xr-x 2 root root   4096 Apr 19 02:55 .
23199751 drwxr-xr-x 1 root root   4096 Apr 19 02:50 ..
 5637314 -rw-r--r-- 1 root root 155744 Apr 20  2018 image10.jpg
 5637324 -rw-r--r-- 1 root root  17248 Apr 20  2018 image1.jpg
 5637326 -rw-r--r-- 1 root root  11641 Apr 20  2018 image2.jpg
 5637328 -rw-r--r-- 1 root root   8750 Apr 20  2018 image3.jpg
 5637320 -rw-r--r-- 1 root root  34622 Apr 20  2018 image4.jpg
 5637322 -rw-r--r-- 1 root root  41637 Apr 20  2018 image5.jpg
 5637318 -rw-r--r-- 1 root root  29093 Apr 20  2018 image6.jpg
 5637316 -rw-r--r-- 1 root root  80785 Apr 20  2018 image7.jpg
 5637306 -rw-r--r-- 1 root root  83248 Apr 20  2018 image8.jpg
 5637312 -rw-r--r-- 1 root root  70520 Apr 20  2018 image9.jpg


In [74]:
!ls -ila 

total 35228
23199751 drwxr-xr-x 1 root root     4096 Apr 19 02:50 .
 5373954 drwxr-xr-x 1 root root     4096 Apr 19 01:53 ..
23199752 drwxr-xr-x 1 root root     4096 Apr  4 20:19 .config
 5637282 -rw-r--r-- 1 root root     6344 Apr 19 01:54 convnet.py
 5637305 drwxr-xr-x 2 root root     4096 Apr 19 02:55 imagens
 5637301 -rw-r--r-- 1 root root   515236 Apr 19 02:54 imagens.zip
 5637302 -rw-r--r-- 1 root root    59261 Apr 19 02:37 imagens.zip.1
 5637303 -rw-r--r-- 1 root root    59261 Apr 19 02:37 imagens.zip.2
 5637304 -rw-r--r-- 1 root root   515236 Apr 19 02:53 images.zip
 5637307 drwxrwxr-x 3 root root     4096 Apr 19 02:47 __MACOSX
 5637299 -rw-r--r-- 1 root root 17430184 Apr 19 02:24 modelo.hdf5
 5637300 -rw-r--r-- 1 root root 17430184 Apr 19 02:33 modeloVGGNet.hdf5
 5637285 drwxr-xr-x 2 root root     4096 Apr 19 01:54 __pycache__
 9306120 drwxr-xr-x 1 root root     4096 Apr  4 20:20 sample_data


In [75]:
# Testando o Modelo Pré-Treinado

# Execute: python cap09-07-test-convnet.py --model modelo/modelo_minivggnet.hdf5 --test-images imagens

# Imports
import cv2
import imutils
import argparse
import numpy as np
from keras.models import load_model
from keras.datasets import cifar10
from imutils import paths
from matplotlib import pyplot as plt
%matplotlib inline

# Hiperparametros
modelo_path='modeloVGGNet.hdf5'
test_images='imagens'
batch_size32=32

# Lista com os labels no CIFAR-10 # 10 classes
gtLabels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# Carregando o modelo
print("\nCarregando o modelo (arquitetura e pesos)...")
model = load_model(modelo_path)

# Testando o modelo em novas imagens
print("\nTestando o modelo em novas imagens")

# Loop pelas imagens de teste
for imagePath in paths.list_images(test_images):
  #print(imagepath)
	# Carrega a imagem, redimensiona para uma resolução fixa de 32 x 32 pixels (ignorando a proporção) e, em seguida, 
	# converte a imagem em ordem RGB para torná-la compatível com a nossa rede
	print("Classificando {}".format(imagePath[imagePath.rfind("/") + 1:]))
	image = cv2.imread(imagePath)
	kerasImage = cv2.resize(image, (32,32))
	kerasImage = cv2.cvtColor(kerasImage, cv2.COLOR_BGR2RGB)
	kerasImage = np.array(kerasImage, dtype="float") / 255.0

	# Adiciona uma dimensão extra à imagem para que possamos passá-la pela rede e, em seguida,
	# fazemos uma previsão na imagem (normalmente faríamos previsões em uma matriz * de imagens, uma a uma)
	kerasImage = kerasImage[np.newaxis, ...]
	probs = model.predict(kerasImage, batch_size=batch_size32)
  print(probs)
  
	prediction = probs.argmax(axis=1)[0]

  print(prediction)
	# Desenha a previsão na imagem de teste 
	#cv2.putText(image, gtLabels[prediction], (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
	#cv2.imshow("Imagem", image)
	#cv2.waitKey(0)

  plt.figure(figsize=(12,8))
  plt.title(gtLabels[prediction])
  plt.imshow(image)


IndentationError: ignored

In [69]:
[x for x in paths.list_images(test_images) ]

['imagens/image2.jpg',
 'imagens/image3.jpg',
 'imagens/image9.jpg',
 'imagens/image7.jpg',
 'imagens/image4.jpg',
 'imagens/image1.jpg',
 'imagens/image6.jpg',
 'imagens/image10.jpg',
 'imagens/image5.jpg',
 'imagens/image8.jpg']

In [8]:
!ls

convnet.py  modelo.hdf5  modeloVGGNet.hdf5  __pycache__  sample_data


In [0]:
!cp modelo.hdf5 modeloVGGNet.hdf5

In [5]:
# Rede Neural Convolucional Para Classificação de Imagens


# import the necessary packages
from convnet import ConvNetFactory
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.datasets import cifar10

# hiperparametros
network='minivggnet'
model_path='modelo.hdf5'
epochs=20
dropout=1
activation="tanh"
batch_size=32
verbose=1

# Carrega os dados de treinamento e teste e, em seguida, dimensiona no intervalo [0, 1]
print("\nCarregando os dados de treino...")
((trainData, trainLabels), (testData, testLabels)) = cifar10.load_data()
trainData = trainData.astype("float") / 255.0
testData = testData.astype("float") / 255.0

# Transforma os rótulos de treinamento e teste em vetores no intervalo [0, numClasses]
# Isso gera um vetor para cada rótulo, onde o índice do rótulo é definido como `1` e todas as outras entradas para `0`
# No caso do CIFAR-10, existem 10 rótulos de classe
trainLabels = np_utils.to_categorical(trainLabels, 10)
testLabels = np_utils.to_categorical(testLabels, 10)

# Coleta os argumentos
kargs = {"dropout": dropout > 0, "activation": activation}

# Treina o modelo usando SGD
print("Compilando o modelo...")
model = ConvNetFactory.build(network, 3, 32, 32, 10, **kargs)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# Inicia o treinamento
print("Iniciando o treinamento...\n")
model.fit(trainData, trainLabels, batch_size=batch_size, epochs=epochs, verbose=verbose)

# Mostra a acurácia nos dados de teste
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=batch_size, verbose=verbose)
print("Acurácia: {:.2f}%".format(accuracy * 100))

# Salva o modelo 
print("\nSalvando o modelo...")
model.save(model_path)


Carregando os dados de treino...
Compilando o modelo...
Iniciando o treinamento...

Epoch 1/20
50000/50000 [==============================] - 16s 314us/step - loss: 1.8898 - acc: 0.4139
Epoch 2/20
50000/50000 [==============================] - 15s 309us/step - loss: 1.2038 - acc: 0.5848
Epoch 3/20
50000/50000 [==============================] - 15s 305us/step - loss: 0.9978 - acc: 0.6543
Epoch 4/20
50000/50000 [==============================] - 16s 321us/step - loss: 0.8859 - acc: 0.6906
Epoch 5/20
50000/50000 [==============================] - 16s 313us/step - loss: 0.8080 - acc: 0.7175
Epoch 6/20
50000/50000 [==============================] - 15s 310us/step - loss: 0.7495 - acc: 0.7370
Epoch 7/20
50000/50000 [==============================] - 16s 314us/step - loss: 0.7099 - acc: 0.7515
Epoch 8/20
50000/50000 [==============================] - 15s 309us/step - loss: 0.6670 - acc: 0.7643
Epoch 9/20
50000/50000 [==============================] - 15s 304us/step - loss: 0.6390 - acc: 0.77

## Faz download do arquivo de pesos do Modelo

In [0]:
from google.colab import files 
model_path='modeloVGGNet.hdf5'
files.download(model_path)